In [1]:
text = "斜め七十七度の並びで泣く泣く嘶くナナハン七台難なく並べて長眺め Naname nanajyuunana-do no narabi de nakunaku inanaku nanahan nanadai nannaku narabete naganagame."
tokens = text.encode("utf-8")
tokens = list(map(int, tokens))

print(tokens)

[230, 150, 156, 227, 130, 129, 228, 184, 131, 229, 141, 129, 228, 184, 131, 229, 186, 166, 227, 129, 174, 228, 184, 166, 227, 129, 179, 227, 129, 167, 230, 179, 163, 227, 129, 143, 230, 179, 163, 227, 129, 143, 229, 152, 182, 227, 129, 143, 227, 131, 138, 227, 131, 138, 227, 131, 143, 227, 131, 179, 228, 184, 131, 229, 143, 176, 233, 155, 163, 227, 129, 170, 227, 129, 143, 228, 184, 166, 227, 129, 185, 227, 129, 166, 233, 149, 183, 231, 156, 186, 227, 130, 129, 32, 78, 97, 110, 97, 109, 101, 32, 110, 97, 110, 97, 106, 121, 117, 117, 110, 97, 110, 97, 45, 100, 111, 32, 110, 111, 32, 110, 97, 114, 97, 98, 105, 32, 100, 101, 32, 110, 97, 107, 117, 110, 97, 107, 117, 32, 105, 110, 97, 110, 97, 107, 117, 32, 110, 97, 110, 97, 104, 97, 110, 32, 110, 97, 110, 97, 100, 97, 105, 32, 110, 97, 110, 110, 97, 107, 117, 32, 110, 97, 114, 97, 98, 101, 116, 101, 32, 110, 97, 103, 97, 110, 97, 103, 97, 109, 101, 46]


In [2]:
def get_stats(tokens):
    counts = {}

    for pair in zip(tokens, tokens[1:]):
        counts[pair] = counts.get(pair, 0) + 1

    return counts
    

In [3]:
def merge(pair, new_token, tokens):
    new_tokens = []

    i = 0
    while i < len(tokens):
        if i < len(tokens) - 1 and (tokens[i], tokens[i + 1]) == pair:
            new_tokens.append(new_token)
            i += 2
        else:
            new_tokens.append(tokens[i])
            i += 1

    return new_tokens

In [4]:
vocab_size = 276
num_merges = vocab_size - 256
old_tokens = tokens

merges = {} # (int, int) -> int
for i in range(num_merges):
    stats = get_stats(tokens)
    top_pair = max(stats, key=stats.get)
    new_token = 256 + i
    print(f"Merging {top_pair} into {new_token}")
    tokens = merge(top_pair, new_token, tokens)
    merges[top_pair] = new_token

Merging (110, 97) into 256
Merging (227, 129) into 257
Merging (32, 256) into 258
Merging (228, 184) into 259
Merging (257, 143) into 260
Merging (227, 131) into 261
Merging (107, 117) into 262
Merging (259, 131) into 263
Merging (263, 229) into 264
Merging (166, 257) into 265
Merging (101, 258) into 266
Merging (256, 262) into 267
Merging (227, 130) into 268
Merging (268, 129) into 269
Merging (259, 265) into 270
Merging (230, 179) into 271
Merging (271, 163) into 272
Merging (272, 260) into 273
Merging (261, 138) into 274
Merging (97, 256) into 275


In [13]:
vocab = { idx: bytes([idx]) for idx in range(256) }
for (a, b), idx in merges.items():
    vocab[idx] = vocab[a] + vocab[b]

def decode(ids):
    tokens = b"".join(vocab[idx] for idx in ids)
    return tokens.decode("utf-8", errors="replace")

In [22]:
def encode(text):
    tokens = list(text.encode("utf-8"))

    while len(tokens) >= 2:
        stats = get_stats(tokens)
        pair = min(stats, key = lambda p: merges.get(p, float("inf")))
        if pair not in merges:
            break
        new_token = merges[pair]
        tokens = merge(pair, new_token, tokens)

    return tokens
        
text = "Nya! Kawaii!"
encoded = encode(text)
decoded = decode(encoded)
print(encoded)
print(decoded)

[78, 121, 97, 33, 32, 75, 97, 119, 97, 105, 105, 33]
Nya! Kawaii!
